# EC Inference

# Setting Up Necessary Things

In [1]:
# Jupyter Notebook Magic Command - Auto Reloading
%reload_ext autoreload
%autoreload 2

# Jupyter Notebook Magic Command - Inline Plotting
%matplotlib inline

In [2]:
# Ignore All Warnings
import warnings
warnings.filterwarnings("ignore")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/All Entertainment Classification

/content/drive/MyDrive/All Entertainment Classification


# Necessary Imports

In [ ]:
! pip install -q transformers[sentencepiece] fastai onnxruntime

In [76]:
import numpy as np

import torch

from transformers import AutoTokenizer

import onnxruntime as rt

# Load Saved Model

In [6]:
model_path = "models/entertainment-category-quantized.onnx"

In [7]:
categories = ['anime', 'book', 'game', 'movie', 'music', 'tv show']

In [9]:
inf_session = rt.InferenceSession('models/entertainment-category-quantized.onnx')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

In [61]:
def entertainment_category(description):
  input_ids = tokenizer(description)['input_ids'][:512]

  probs = inf_session.run([output_name], {input_name: [input_ids]})[0]

  mask = np.where(probs[0] == probs.max())[0][0]

  return categories[mask], torch.sigmoid(torch.FloatTensor(probs))[0], torch.sigmoid(torch.FloatTensor(probs))[0].max()

# Inference Varification

## Inference - 1: Movie

In [72]:
# Movie: The Godfather
desc = "Don Vito Corleone, head of a mafia family, decides to hand over his empire to his youngest son Michael. However, his decision unintentionally puts the lives of his loved ones in grave danger."

In [73]:
entertainment_category(desc)

('movie',
 tensor([0.1648, 0.1812, 0.0822, 0.9946, 0.0566, 0.9528]),
 tensor(0.9946))

## Inference - 2: Music

In [74]:
# Music: Live Fast Die Young
desc = "When I wake up, I look into the mirror, I can see a clearer, vision, I should start living today, Cause today is gonna be the day, is gonna be the day, Cause today is gonna be the day, is gonna be the day"

In [75]:
entertainment_category(desc)

('music',
 tensor([0.2354, 0.2064, 0.1537, 0.5060, 0.9985, 0.1632]),
 tensor(0.9985))

## Inference - 3: Anime

In [79]:
# Anime: One Piece
desc = "Gol D. Roger was known as the 'Pirate King', the strongest and most infamous being to have sailed the Grand Line. The capture and execution of Roger by the World Government brought a change throughout the world. His last words before his death revealed the existence of the greatest treasure in the world, One Piece. It was this revelation that brought about the Grand Age of Pirates, men who dreamed of finding One Piece—which promises an unlimited amount of riches and fame—and quite possibly the pinnacle of glory and the title of the Pirate King."

In [80]:
entertainment_category(desc)

('anime',
 tensor([0.9906, 0.6193, 0.1797, 0.0424, 0.0669, 0.8937]),
 tensor(0.9906))

## Inference - 4: Game

In [81]:
# Game: March Of Soldiers
desc = "March Of Soldiers is a real time strategy single player , It is a military game based on the player's skill and the strength of his financial economy"

In [82]:
entertainment_category(desc)

('game',
 tensor([0.2049, 0.0707, 0.9988, 0.3088, 0.1598, 0.2501]),
 tensor(0.9988))